<a href="https://colab.research.google.com/github/nilanahar/MMAI_894_DeepLearning_Project_TeamRosedale/blob/main/MMAI_894_Team_Project_v1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up Environment

#### Packages to Load

In [3]:
from google.colab import drive
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
import os
import re

#### Set Directories

In [9]:
# define my google drive
drive.mount('/content/drive')

# Define the file path (adjust it to match your folder structure)
training_mcq_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Training/MultipleChoice_abstract_v002_train2015_questions.json"
training_oeq_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Training/OpenEnded_abstract_v002_train2015_questions.json"
training_answers_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Training/abstract_v002_train2015_annotations.json"
training_images_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Training/scene_img_abstract_v002_train2015"

validation_mcq_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Validation/MultipleChoice_abstract_v002_val2015_questions.json"
validation_oeq_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Validation/OpenEnded_abstract_v002_val2015_questions.json"
validation_answers_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Validation/abstract_v002_val2015_annotations.json"
validation_images_file_path = "/content/drive/MyDrive/Colab Notebooks/894/Validation/scene_img_abstract_v002_val2015"

# testing_mcq_file_path = "/content/drive/MyDrive/Smith School of Business (MMAI)/894 Project/Questions_Train_abstract_v002/MultipleChoice_abstract_v002_train2015_questions.json"
# testing_oeq_file_path = "/content/drive/MyDrive/Smith School of Business (MMAI)/894 Project/Questions_Train_abstract_v002/OpenEnded_abstract_v002_train2015_questions.json"
# testing_answers_file_path = "/content/drive/MyDrive/Smith School of Business (MMAI)/894 Project/abstract_v002_train2015_annotations.json"
# testing_images_file_path = "/content/drive/MyDrive/Smith School of Business (MMAI)/894 Project/scene_img_abstract_v002_train2015"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Functions to Use as You Go

In [10]:
# Function to load multiple JSON data
def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

def extract_qna_features(mcq_file_path, oeq_file_path, answers_file_path):
    # Now you can use this function to load multiple JSON files
    mcq_json_data = load_json(mcq_file_path)
    oeq_json_data = load_json(oeq_file_path)
    answers_json_data = load_json(answers_file_path)

    # convert 3 json data into 3 pandas dataframe (tabular form)
    mcq_df = pd.json_normalize(mcq_json_data['questions'])[['image_id', 'question_id', 'question', 'multiple_choices']]
    oeq_df = pd.json_normalize(oeq_json_data['questions'])[['image_id', 'question_id', 'question']]
    answers_df = pd.json_normalize(answers_json_data['annotations'])[['image_id', 'question_id', 'question_type', 'answers', 'multiple_choice_answer', 'answer_type']]

    # Create a unique identifier for each row to track back after transformations
    answers_df['index'] = answers_df.index
    # Explode the 'answers' column into separate rows, while keeping other columns intact
    answers_expanded_df = answers_df.explode('answers')
    # normalization as before
    answers_details = pd.json_normalize(answers_expanded_df['answers'])
    answers_details['index'] = answers_expanded_df.index
    # Concatenate back to the actual answers DataFrame
    answers_expanded_df = pd.merge(answers_df, answers_details, on='index', how='left')
    answers_merged_df = answers_expanded_df[['image_id', 'question_id', 'question_type',
                                              'multiple_choice_answer', 'answer_type',
                                              'answer', 'answer_id', 'answer_confidence',
                                              'index']]
    answers_merged_df = answers_merged_df.rename(columns={'multiple_choice_answer': 'mcq_answer',
                                                              'answer_type': 'mcqa_type',
                                                              'answer': 'oeq_answer',
                                                              'answer_id': 'oeqa_id',
                                                              'answer_confidence': 'oeqa_confidence'
                                                              })

    questions_merged_df = pd.merge(mcq_df, oeq_df, on=['image_id', 'question_id'], how='left', suffixes=('_oeq', '_mcq'))
    final_qna_df = pd.merge(questions_merged_df, answers_merged_df, on=['image_id', 'question_id'], how='left')
    return mcq_df, oeq_df, answers_merged_df, final_qna_df


# Function to extract last five digits from file names
def extract_last_five_digits(filename):
    match = re.search(r'(\d{5})\.png$', filename)
    if match:
        return match.group(1)
    else:
        raise ValueError(f"Filename {filename} does not match the expected pattern.")

def extract_image_features(model, images_file_path):
    images_features_dict = {}
    for img_name in tqdm(os.listdir(images_file_path)):
        img_path = os.path.join(images_file_path, img_name)

        # Extract image_id (adjust slice according to your filename structure)
        image_id = img_name[-9:-4]  # Example: extracting last 5 digits before file extension

        # Load and preprocess the image
        img = image.load_img(img_path, target_size=(224, 224))
        x = preprocess_input(np.expand_dims(image.img_to_array(img), axis=0))

        # Extract features and store them in the dictionary
        features = model.predict(x)
        images_features_dict[image_id] = features.flatten()

    images_features_df = pd.DataFrame.from_dict(images_features_dict, orient='index')
    images_features_df.reset_index(inplace=True)
    images_features_df.rename(columns={'index': 'image_id'}, inplace=True)

    return images_features_df


def complete_datasets(qna_df, images_features_df):
    qna_df = qna_df.sort_values(by='image_id').reset_index(drop=True)
    images_features_df = images_features_df.sort_values(by='image_id').reset_index(drop=True)
    images_features_df['image_id'] = images_features_df['image_id'].astype(int)
    # Merge the 3 dataframes into one (base dataset is image_df (unique), left join qna on image id)
    complete_training_set = pd.merge(images_features_df, qna_df, on='image_id', how='left')
    return complete_training_set


#### Define Constants

In [6]:
### Load the pre-trained model that you wish to use to extract image features ###
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


# Step 1: Data Collection

### 1.1 Training Datasets

#### 1.1.1 Extracting Questions & Answers

In [11]:
train_mcq_df, train_oeq_df, train_answers_merged_df, train_final_qna_df = extract_qna_features(training_mcq_file_path, training_oeq_file_path, training_answers_file_path)
train_final_qna_df.head()

,image_id,question_id,question_oeq,multiple_choices,question_mcq,question_type,mcq_answer,mcqa_type,oeq_answer,oeqa_id,oeqa_confidence,index
0,11779,117792,Who looks happier?,"[alive, 1, woman, purple, 2, yes, white, boy, ...",Who looks happier?,who,man,other,old person,1,maybe,0
1,11779,117792,Who looks happier?,"[alive, 1, woman, purple, 2, yes, white, boy, ...",Who looks happier?,who,man,other,man,2,maybe,0
2,11779,117792,Who looks happier?,"[alive, 1, woman, purple, 2, yes, white, boy, ...",Who looks happier?,who,man,other,man,3,yes,0
3,11779,117792,Who looks happier?,"[alive, 1, woman, purple, 2, yes, white, boy, ...",Who looks happier?,who,man,other,man,4,yes,0
4,11779,117792,Who looks happier?,"[alive, 1, woman, purple, 2, yes, white, boy, ...",Who looks happier?,who,man,other,old man,5,yes,0


##### Save Workspace

In [ ]:
# Save the dataframes to Google Drive
joblib.dump(train_final_qna_df, "/content/drive/MyDrive/Colab Notebooks/894/Training/questions_answers_features.joblib")

#### 1.1.2 Extracting Images' Features


In [ ]:
train_images_features_df = extract_image_features(model, training_images_file_path)

##### Save Workspace

In [ ]:
joblib.dump(train_images_features_df, "/content/drive/MyDrive/Colab Notebooks/894/Training/images_features.joblib")

#### 1.1.3 Complete Training Dataset

##### Load Workspace

In [12]:
train_qna_df = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Training/questions_answers_features.joblib")
train_images_df = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Training/images_features.joblib")
train_images_df.shape

(20000, 2049)

In [13]:
train_complete_df = complete_datasets(train_qna_df, train_images_df)
train_complete_df.head()

,image_id,0,1,2,3,4,5,6,7,8,...,question_id,question_oeq,question_mcq,question_type,mcq_answer,mcqa_type,oeq_answer,oeqa_id,oeqa_confidence,index
0,0,0.037811,1.084643,0.125582,2.314969,0.325828,0.302625,0.976951,0.753177,0.227697,...,0,What color is the ladies pants?,What color is the ladies pants?,what color is the,tan,other,brown,7,yes,13752
1,0,0.037811,1.084643,0.125582,2.314969,0.325828,0.302625,0.976951,0.753177,0.227697,...,0,What color is the ladies pants?,What color is the ladies pants?,what color is the,tan,other,beige,8,yes,13752
2,0,0.037811,1.084643,0.125582,2.314969,0.325828,0.302625,0.976951,0.753177,0.227697,...,0,What color is the ladies pants?,What color is the ladies pants?,what color is the,tan,other,tan,9,yes,13752
3,0,0.037811,1.084643,0.125582,2.314969,0.325828,0.302625,0.976951,0.753177,0.227697,...,0,What color is the ladies pants?,What color is the ladies pants?,what color is the,tan,other,brown,10,yes,13752
4,0,0.037811,1.084643,0.125582,2.314969,0.325828,0.302625,0.976951,0.753177,0.227697,...,1,How is the equipments with bars called?,How is the equipments with bars called?,how,monkey bars,other,monkey bars,1,yes,13753


##### Save Workspace

In [ ]:
joblib.dump(train_complete_df, "/content/drive/MyDrive/Colab Notebooks/894/Training/complete_training_set_tabular.joblib")

### 1.2 Validation Datasets

#### 1.2.1 Extracting Questions & Answers


In [15]:
valid_mcq_df, valid_oeq_df, valid_answers_merged_df, valid_final_qna_df = extract_qna_features(validation_mcq_file_path, validation_oeq_file_path, validation_answers_file_path)
valid_final_qna_df.head()

,image_id,question_id,question_oeq,multiple_choices,question_mcq,question_type,mcq_answer,mcqa_type,oeq_answer,oeqa_id,oeqa_confidence,index
0,27578,275780,Is the dog asleep?,"[white, picnicking, yes, hot, dog and girl, bl...",Is the dog asleep?,is the dog,yes,yes/no,yes,1,yes,0
1,27578,275780,Is the dog asleep?,"[white, picnicking, yes, hot, dog and girl, bl...",Is the dog asleep?,is the dog,yes,yes/no,yes,2,maybe,0
2,27578,275780,Is the dog asleep?,"[white, picnicking, yes, hot, dog and girl, bl...",Is the dog asleep?,is the dog,yes,yes/no,yes,3,yes,0
3,27578,275780,Is the dog asleep?,"[white, picnicking, yes, hot, dog and girl, bl...",Is the dog asleep?,is the dog,yes,yes/no,yes,4,yes,0
4,27578,275780,Is the dog asleep?,"[white, picnicking, yes, hot, dog and girl, bl...",Is the dog asleep?,is the dog,yes,yes/no,yes,5,yes,0


##### Save Workspace

In [16]:
joblib.dump(valid_final_qna_df, "/content/drive/MyDrive/Colab Notebooks/894/Validation/questions_answers_features.joblib")

['/content/drive/MyDrive/Colab Notebooks/894/Validation/questions_answers_features.joblib']

#### 1.2.2 Extracting Images' Features

In [ ]:
valid_images_features_df = extract_image_features(model, validation_images_file_path)

  0%|          | 0/10000 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


  0%|          | 1/10000 [00:06<17:24:48,  6.27s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


  0%|          | 2/10000 [00:07<8:39:20,  3.12s/it] 

...

  4%|▍         | 394/10000 [07:30<2:33:05,  1.05it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


  4%|▍         | 395/10000 [07:31<2:49:33,  1.06s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


  4%|▍         | 396/10000 [07:32<2:44:15,  1.03s/it]

##### Save Workspace

In [ ]:
joblib.dump(valid_images_features_df, "/content/drive/MyDrive/Colab Notebooks/894/Validation/images_features.joblib")

#### 1.2.3  Complete Validation Dataset

In [ ]:
train_qna_df = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Training/questions_answers_features.joblib")
train_images_df = joblib.load("/content/drive/MyDrive/Colab Notebooks/894/Training/images_features.joblib")
train_images_df.shape

### 1.3 Testing Datasets

##### 1.3.1 Extracting Q & As

##### 1.3.2 Extracting Images' Features

#### 1.3.3 Complete Testing Set

## Overall Information (maybe useless）

In [ ]:
def summarize_dataset(data, df):
    print("=== Dataset Summary ===\n")

    # 1. Dataset Info
    print("1. Dataset Info:")
    for key, value in data['info'].items():
        print(f"   {key}: {value}")
    print("\n")

    # 2. Data Type and Subtype
    print("2. Data Type and Subtype:")
    print(f"   Data Type: {data['data_type']}")
    print(f"   Data Subtype: {data['data_subtype']}\n")

    # 3. License Information
    print("3. License Information:")
    for key, value in data['license'].items():
        print(f"   {key}: {value}")
    print("\n")

    # 4. Annotations Overview
    print("4. Annotations Overview:")
    print(f"   Total Annotations: {len(data['annotations'])}")
    print(f"   Total Entries in DataFrame: {len(df)}\n")

    # 5. Unique Question Types
    print("5. Unique Question Types:")
    print(f"   {df['question_type'].nunique()} types")
    print(f"   {df['question_type'].unique()}\n")

    # 6. Unique Answer Types
    print("6. Unique Answer Types:")
    print(f"   {df['answer_type'].nunique()} types")
    print(f"   {df['answer_type'].unique()}\n")

    # 7. Answer Confidence Levels
    print("7. Answer Confidence Levels:")
    print(df['answer_confidence'].value_counts())
    print("\n")

    # 8. Sample Annotation
    print("8. Sample Annotation:")
    print(json.dumps(data['annotations'][0], indent=4))

summarize_dataset(data, df)


## Basic Data Information

In [ ]:
# View data dimensions
print("Data dimensions:", df.shape)

# View data overview
print(df.info())

# Check for missing values
print(df.isnull().sum())

## Distribution of Question Types

In [ ]:
# Count the occurrences of each question type
question_type_counts = df['question_type'].value_counts()

# Visualization
plt.figure(figsize=(20,15))
plt.subplots_adjust(left=0.5)
sns.countplot(y='question_type', data=df, order=question_type_counts.index)
plt.title('Distribution of Question Types')
plt.xlabel('Count')
plt.ylabel('Question Type')
# Your plotting code here
plt.show()

print(question_type_counts)

## Distribution of Answer Types

In [ ]:
# Count the occurrences of each answer type
answer_type_counts = df['answer_type'].value_counts()

# Visualization
plt.figure(figsize=(6,6))
answer_type_counts.plot.pie(autopct='%1.1f%%')
plt.title('Distribution of Answer Types')
plt.ylabel('')
plt.show()


## Distribution of Answer Confidence

In [ ]:
# Count the answer confidence levels
confidence_counts = df['answer_confidence'].value_counts()

# Visualization
plt.figure(figsize=(6,4))
sns.countplot(x='answer_confidence', data=df, order=['yes', 'maybe', 'no'])
plt.title('Distribution of Answer Confidence')
plt.xlabel('Confidence Level')
plt.ylabel('Count')
plt.show()


## Most Common Answers

In [ ]:
# Count the occurrences of each answer
answer_counts = df['answer'].value_counts()

# Display the top 10 most common answers
print("Top 10 most common answers:")
print(answer_counts.head(10))

# Visualization
plt.figure(figsize=(12,6))
sns.barplot(x=answer_counts.head(10).index, y=answer_counts.head(10).values)
plt.title('Top 10 Most Common Answers')
plt.xlabel('Answer')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()


# Data Preprocessing

## Handling Missing Values

In [ ]:
# Drop rows with missing values
df = df.dropna()

# Encoder

In [ ]:
# One-hot encoding for categorical variables
df_encoded = pd.get_dummies(df, columns=['answer_confidence', 'answer_type', 'question_type'])

## Process Text Data


In [ ]:
import re
import string

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove whitespace
    text = text.strip()
    return text

df['answer_processed'] = df['answer'].apply(preprocess_text)

# Model Architecture

## Image Feature Extraction

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model

# Load pretrained ResNet50 model + higher level layers
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Freeze base model
base_model.trainable = False


## Text Processing

In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assume max_question_length and vocab_size are defined
question_input = Input(shape=(max_question_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=300)(question_input)
lstm_layer = LSTM(256)(embedding_layer)


## Feature Fusion

In [ ]:
from tensorflow.keras.layers import concatenate, Dense

# Image features from ResNet50
image_features = base_model.output

# Combine image and text features
combined_features = concatenate([image_features, lstm_layer])

# Add Dense layers
dense_layer = Dense(256, activation='relu')(combined_features)
dense_layer = Dense(128, activation='relu')(dense_layer)


## Output Layer

In [ ]:
# Assuming num_answers is the number of possible answers
output_layer = Dense(num_answers, activation='softmax')(dense_layer)


## Compile the Model

In [ ]:
from tensorflow.keras.models import Model

model = Model(inputs=[base_model.input, question_input], outputs=output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
